# Finding the closest stable pose

This notebook provides an example of how to find the closest stable pose $T_\text{part}^\text{tag}(x, y, \gamma)$ to a given pose estimate $\widehat{T}_\text{part}^\text{tag}$ . It does this by formulating a nonlinear least-squares minimization problem that is solved by Levenberg-Marquardt using [SymForce](https://github.com/symforce-org/symforce). (This avoids the need to code a solver by ourselves, and in particular avoids the need to derive Jacobians by ourselves — both of these were done in the example code for extrinsic calibration that I shared with you earlier.)

To run this notebook, you need a conda environment with SymForce, for example one that is created as follows:

```
mamba create -n sf-test
mamba activate sf-test
mamba install python=3 numpy scipy sympy matplotlib notebook ipywidgets
pip install symforce
```

(I am using `mamba` instead of `conda`, and have installed everything with [`miniforge`](https://github.com/conda-forge/miniforge) instead of `miniconda`. If you like, you can use `conda`/`miniconda` instead.)

In hindsight, after looking at the results in this notebook, it is clear to me that — for the particular way in which we parameterize stable poses — "replacing the $z$ coordinate" is an equivalent way to generate the position of the closest stable pose (that is, the values of $x$ and $y$ produced by optimization are always the same as $x_\text{est}$ and $y_\text{est}$). It is likely that there is an analytical solution for $\gamma$ as well — again, for the particular way in which we parameterize stable poses. But, I would still use the method I propose here.

### Create nonlinear least squares problem to be solved by Levenberg-Marquardt

Initialize [symforce](https://github.com/symforce-org/symforce) with a nonzero epsilon to prevent singularities.

In [1]:
import symforce
symforce.set_epsilon_to_symbol()

Import everything else we need.

In [2]:
import symforce.symbolic as sf
import numpy as np
from symforce.values import Values
from symforce.opt.factor import Factor
from symforce.opt.optimizer import Optimizer
from symforce import codegen
import sym
import matplotlib.pyplot as plt

Create a symbolic function that maps $x, y, \gamma$ to a stable pose.

**THIS IS ONLY A SIMPLE EXAMPLE! YOU WOULD NEED TO MODIFY THIS FUNCTION SO IT ACTUALLY USES THE PARAMETERIZATION YOU DERIVED!**

In [3]:
def construct_stable_pose(x: sf.Scalar, y: sf.Scalar, gamma: sf.Scalar) -> sf.Pose3:
    return sf.Pose3(
        R=sf.Rot3.from_yaw_pitch_roll(gamma, 0, 0),
        t=sf.V3(x, y, 0),
    )

Create a numeric version of this function.

In [4]:
# construct_stable_pose_numeric = codegen.Codegen.function(
#     func=construct_stable_pose,
#     config=codegen.PythonConfig(),
# ).lambdify()

construct_stable_pose_numeric = construct_stable_pose

AttributeError: 'Codegen' object has no attribute 'lambdify'

Create a function that computes the (weighted) error between a stable pose and a pose estimate.

In [ ]:
def pose_residual(
    # Things we want to find
    x: sf.Scalar,
    y: sf.Scalar,
    gamma: sf.Scalar,
    # Things we are given
    T_est: sf.Pose3,
    w: sf.Scalar,
    # Things that are details of symforce
    epsilon: sf.Scalar,
) -> sf.V4:
    # Create stable pose
    T_sta = construct_stable_pose(x, y, gamma)
    # Find difference in position
    t_diff = T_est.t - T_sta.t
    # Find difference in angle
    a_diff = T_est.R.angle_between(T_sta.R)
    # Return error (could be weighted)
    return sf.V4(t_diff.x, t_diff.y, t_diff.z, w * a_diff)

Create factors (i.e., the list of all residuals).

In [ ]:
factors = [
    Factor(
        residual=pose_residual,
        keys=[
            'x',
            'y',
            'gamma',
            'T_est',
            'w',
            'epsilon',
        ]
    )
]

Create optimizer.

In [ ]:
optimizer = Optimizer(
    factors=factors,
    optimized_keys=[
        'x',
        'y',
        'gamma',
    ],
    debug_stats=True,
)

### Example in which the pose estimate is a stable pose

Choose a stable pose.

In [ ]:
x_est = -0.1
y_est = 0.4
gamma_est = 0.2

Create initial values.

In [ ]:
initial_values = Values(
    # Initial guess of stable pose
    x=0.,
    y=0.,
    gamma=0.,
    # Pose estimate
    T_est=construct_stable_pose_numeric(x_est, y_est, gamma_est),
    # Weight on angle error
    w=1.,
    # Epsilon
    epsilon=sym.epsilon,
)

Run the optimizer.

In [ ]:
result = optimizer.optimize(initial_values)

Check for success.

In [ ]:
print(result.status)

Check final error.

In [ ]:
print(result.error())

Get poses.

In [ ]:
# Pose estimate (this was given and was not changed by the optimizer)
T_est = result.optimized_values['T_est']

# Stable pose that is closest to the pose estimate
T_sta = construct_stable_pose_numeric(
    result.optimized_values['x'],
    result.optimized_values['y'],
    result.optimized_values['gamma'],
)

Create helper functions to draw results.

In [ ]:
def draw_frame(ax, R, t, s=1., linewidth=1., linestyle='-'):
    o = t + R @ np.array([0., 0., 0.])
    x = t + R @ np.array([s, 0., 0.])
    y = t + R @ np.array([0., s, 0.])
    z = t + R @ np.array([0., 0., s])
    ax.plot(o[0], o[1], o[2], 'k.')
    ax.plot([o[0], x[0]], [o[1], x[1]], [o[2], x[2]], 'r', linewidth=linewidth, linestyle=linestyle)
    ax.plot([o[0], y[0]], [o[1], y[1]], [o[2], y[2]], 'g', linewidth=linewidth, linestyle=linestyle)
    ax.plot([o[0], z[0]], [o[1], z[1]], [o[2], z[2]], 'b', linewidth=linewidth, linestyle=linestyle)

def draw_view(ax, elev, azim, roll, T_sta, T_est, s=1.):
    ax.set_proj_type('ortho')
    
    # Pose estimate
    draw_frame(
        ax,
        T_est.R.to_rotation_matrix(),
        T_est.t,
        s=s,
    )
    
    # Stable pose that is closest to pose estimate
    draw_frame(
        ax,
        T_sta.R.to_rotation_matrix(),
        T_sta.t,
        s=s,
        linewidth=3.,
        linestyle='--',
    )
    
    ax.set_xlim(-1, 1)
    ax.set_ylim(-1, 1)
    ax.set_zlim(0, 2)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    ax.set_xticks([-1, 0, 1])
    ax.set_yticks([-1, 0, 1])
    ax.set_zticks([0, 1, 2])
    
    ax.set_aspect('equal')
    ax.view_init(elev=elev, azim=azim, roll=roll)

def draw_results(T_sta, T_est, s=1.):
    fig, (ax, ax_x, ax_y, ax_z) = plt.subplots(1, 4, figsize=(20, 60), subplot_kw={'projection': '3d'})
    draw_view(ax, 15., 30., 0., T_sta, T_est, s=s)
    draw_view(ax_x, 0., -90., 0., T_sta, T_est, s=s)
    draw_view(ax_y, 0., 0., 0., T_sta, T_est, s=s)
    draw_view(ax_z, 90., -90., 0., T_sta, T_est, s=s)
    fig.tight_layout()
    plt.show()

Draw poses (should only see one, because both the pose estimate and the closest stable pose should be identical).

In [ ]:
draw_results(T_sta, T_est, s=0.5)

Compare parameters of closest stable pose to parameters of pose estimate (should be the same).

In [ ]:
x_sta = result.optimized_values['x']
y_sta = result.optimized_values['y']
gamma_sta = result.optimized_values['gamma']

print(f'{x_est:7.4f}, {y_est:7.4f}, {gamma_est:7.4f}   (pose estimate)')
print(f'{x_sta:7.4f}, {y_sta:7.4f}, {gamma_sta:7.4f}   (closest stable pose)')

### Example in which the pose estimate is not a stable pose

Create initial values.

In [ ]:
initial_values = Values(
    # Initial guess of stable pose
    x=0.,
    y=0.,
    gamma=0.,
    # Pose estimate
    T_est=sym.Pose3(
        R=sym.Rot3.from_yaw_pitch_roll(0.9, 0., 0.),
        t=np.array([0.2, -0.1, 0.4]),
    ),
    # Weight on angle error
    w=1.,
    # Epsilon
    epsilon=sym.epsilon,
)

Run the optimizer.

In [ ]:
result = optimizer.optimize(initial_values)

Check for success.

In [ ]:
print(result.status)

Check final error.

In [ ]:
print(result.error())

Get poses.

In [ ]:
# Pose estimate (this was given and was not changed by the optimizer)
T_est = result.optimized_values['T_est']

# Stable pose that is closest to the pose estimate
T_sta = construct_stable_pose_numeric(
    result.optimized_values['x'],
    result.optimized_values['y'],
    result.optimized_values['gamma'],
)

Draw poses (should have the same orientation, with one below the other).

In [ ]:
draw_results(T_sta, T_est, s=0.5)

### Another example in which the pose estimate is not a stable pose

Create initial values.

In [ ]:
initial_values = Values(
    # Initial guess of stable pose
    x=0.,
    y=0.,
    gamma=0.,
    # Pose estimate
    T_est=sym.Pose3(
        R=sym.Rot3.from_yaw_pitch_roll(-0.2, 0.5, 0.8),
        t=np.array([-0.3, 0.2, 0.7]),
    ),
    # Weight on angle error
    w=1.,
    # Epsilon
    epsilon=sym.epsilon,
)

Run the optimizer.

In [ ]:
result = optimizer.optimize(initial_values)

Check for success.

In [ ]:
print(result.status)

Check final error.

In [ ]:
print(result.error())

Get poses.

In [ ]:
# Pose estimate (this was given and was not changed by the optimizer)
T_est = result.optimized_values['T_est']

# Stable pose that is closest to the pose estimate
T_sta = construct_stable_pose_numeric(
    result.optimized_values['x'],
    result.optimized_values['y'],
    result.optimized_values['gamma'],
)

Draw poses (should have the same orientation, with one below the other).

In [ ]:
draw_results(T_sta, T_est, s=0.5)